In [1]:
import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm_notebook as tqdm
import time

# function

In [2]:
def load_data(path,label):
    data = pd.read_csv(path,header=None)
    data['label'] = label
    return data

In [3]:
def load_train():
    df = pd.DataFrame()
    for label in range(10):
        path = 'digit/digit_train{0}.csv'.format(label)
        df_temp = load_data(path,label)
        df = pd.concat([df,df_temp])
    X = df.drop(['label'],axis=1)
    y = df['label']
    return (X.values,y.values)

def load_test():
    df = pd.DataFrame()
    for label in range(10):
        path = 'digit/digit_test{0}.csv'.format(label)
        df_temp = load_data(path,label)
        df = pd.concat([df,df_temp])
    X = df.drop(['label'],axis=1)
    y = df['label']
    return (X.values,y.values)

In [4]:
def meshgrid_ij(X):
    X = np.array([X])
    shape = X.shape
    X_T = X.reshape((shape[1],shape[0],shape[2]))
    return np.broadcast_arrays(X,X_T)

In [5]:
def get_K_matrix(X,h=1):
    XX,YY = meshgrid_ij(X)
    n = XX.shape[0]
    K = [[np.exp(-np.sum((XX[i,j,:]-YY[i,j,:])**2)/2/h**2) for i in range(n)] for j in range(n)]
    return K

In [76]:
def predict(x,theta,X_train,h=1):
    predict = np.sum(theta.T*np.exp(-np.sum((x-X_train)**2
                                                 ,axis=1)/2/1**2)
                     ,axis=1)
    return np.argmax(predict)

In [80]:
def get_acc(pred,test):
    n = len(pred)
    return sum(pred==test)/n

# dataset

In [69]:
X_train,label_train = load_train()
X_test,label_test = load_test()
y_train = pd.get_dummies(label_train, sparse=True).values
y_test = pd.get_dummies(label_test, sparse=True).values

# predict

In [9]:
%%time
K = get_K_matrix(X_train,)

CPU times: user 4min 52s, sys: 1.29 s, total: 4min 53s
Wall time: 4min 55s


In [27]:
%%time
# calculate theta
theta = np.linalg.inv(K)@y_train

In [77]:
label_pred = []
for x in tqdm(X_test):
    label_pred.append(predict(x,theta,X_train))

In [83]:
acc = get_acc(label_pred,label_test)
print('accuracy: {0}%'.format(acc*100))

accuracy: 96.5%
